# Working with Landsat Thematic Mapper Imagery over Time


![](http://esri.github.io/arcgis-python-api/notebooks/nbimages/02_change_detection_app_01.gif)

# Questions
- How does land change manifest itself in time-series of multispectral imagery?
- Can you identify when a significant disturbance occurred?

# Let's explore Landsat TM Data over a period of 30 years!

- About 20 years ago, my house was on farmland 
- Can we use the 30 years of TM data to identify when development occurred? 

# Let's log into Portal to and access our Landsat TM NDVI service

In [2]:
from arcgis.features import SpatialDataFrame
from arcgis.raster.functions import *
from arcgis.raster import ImageryLayer
from arcgis.geometry import Geometry
from arcgis.geometry import Point
from arcgis.gis import GIS

gis = GIS('http://www.arcgis.com')

In [4]:
ndvi_svc ="https://fedciv.esri.com/imageserver/rest/services/LandsatTM_NDVI/ImageServer"
ndvi_lyr = ImageryLayer(ndvi_svc, gis=gis)

In [13]:
ms_svc = "https://landsat2.arcgis.com/arcgis/rest/services/LandsatGLS/MS/ImageServer"
ms_lyr = ImageryLayer(ms_svc, gis=gis)

# O'Fallon, MO

In [4]:
geometry = Point({"x" :-10093991.604, "y" : 4689459.491, "spatialReference" : {"wkid" : 102100}})

In [5]:
map1 = gis.map("122 Arabian Path, O'Fallon, MO, USA")
map1

MapView(layout=Layout(height='400px', width='100%'))

In [6]:
map1.basemap = 'satellite'

In [7]:
symbol = {"color":[128,0,0,128],
                    "size":18,
                    "angle":0,
                    "xoffset":0,
                    "yoffset":0,
                    "type":"esriSMS",
                    "style":
                    "esriSMSCircle",
                    "outline":
                        {"color":[128,0,0,255],
                         "width":1,
                         "type":"esriSLS",
                         "style":"esriSLSSolid"}
}

In [8]:
map1.draw(geometry, symbol=symbol)

In [9]:
map1.zoom = 14

# How many rasters are over O'Fallon, MO?

## Let's get the table as a dataframe and query all the pixels

In [14]:
from arcgis.geometry import filters

geometry = Point({"x" :-10093991.604, "y" : 4689459.491, "spatialReference" : {"wkid" : 102100}})

sp_filter = filters.intersects(geometry=geometry)

im_sdf = ms_lyr.query(geometry_filter=sp_filter, return_all_records=True).df

In [15]:
len(im_sdf)

57

In [16]:
im_sdf.head(3)

,AcquisitionDate,Best,Category,CenterX,CenterY,CloudCover,DateUpdated,DayOfYear,GroupName,HighPS,...,ProductName,SensorName,Shape_Area,Shape_Length,SunAzimuth,SunElevation,WRS_Path,WRS_Row,dataset_id,SHAPE
0,1277942400000,72976033,1,-1.010793e+07,4.707152e+06,0.0,None,182.0,L5024033_03320100701,120.0,...,L1T,LANDSAT-5-TM,5.364016e+10,9.275399e+05,121.3225,64.44746,24.0,33.0,GLS-2010,"{'rings': [[[-9962833.3609, 4792706.126100004]..."
1,1262304000000,112000000,2,-1.005351e+07,4.319069e+06,NaN,None,NaN,Ov_i04_L01_R00000016_C00000006GLS-2010,300.0,...,,None,2.359296e+12,6.144000e+06,NaN,NaN,NaN,NaN,GLS-2010,"{'rings': [[[-9285507.0672, 3551069.3966000006..."
2,1262304000000,112000000,2,-9.285507e+06,5.087069e+06,NaN,None,NaN,Ov_i04_L02_R0000000B_C00000003GLS-2010,600.0,...,,None,9.437184e+12,1.228800e+07,NaN,NaN,NaN,NaN,GLS-2010,"{'rings': [[[-7749507.0671999995, 3551069.3966..."


# Let's turn Unix time into a date\time

In [18]:
import pandas as pd

im_sdf['Date'] =  pd.to_datetime(im_sdf['AcquisitionDate'], unit='ms')
im_sdf['Date'].tail()

52   1975-01-01
53   1975-01-01
54   1975-01-01
55   1976-09-11
56   1979-09-24
Name: Date, dtype: datetime64[ns]

# Let's Look at Some Images

In [10]:
map_old = gis.map("122 Arabian Path, O'Fallon, MO")
#map_old

In [11]:
map_new = gis.map("122 Arabian Path, O'Fallon, MO")
#map_new

In [17]:
map_new.zoom = 14
map_old.zoom = 14

In [13]:
im_sdf[im_sdf['CloudCover']<.10].Date.head()

4    1982-11-17 16:10:16.657
5    1983-01-04 16:10:48.224
10   1984-10-13 16:12:09.654
32   1988-07-20 16:12:47.393
34   1988-12-27 16:12:23.535
Name: Date, dtype: datetime64[ns]

In [14]:
im_sdf[im_sdf['CloudCover']<.10].Name.head()

4     LT40240331982321XXX04
5     LT40240331983004XXX08
10    LT50240331984287XXX03
32    LT50240331988202XXX03
34    LT50240331988362XXX02
Name: Name, dtype: object

In [16]:
selected_oldest = ms_lyr.filter_by(where="Name = 'LT40240331982321XXX04'")
selected_newest = ms_lyr.filter_by(where="Name = 'LT50240332010102EDC00'")

map_old.add_layer(stretch(extract_band(selected_oldest,[2,1,0]),
                         stretch_type='StdDev',
                         num_stddev=2.5,
                         dra=True))

map_new.add_layer(stretch(extract_band(selected_newest,[2,1,0]),
                         stretch_type='StdDev',
                         num_stddev=2.5,
                         dra=True))

In [22]:
map_old.draw(geometry, symbol=symbol)
map_new.draw(geometry, symbol=symbol)

In [18]:
#import ipywidgets as widgets
#
#tab = widgets.Tab([map_old, map_new])
#tab.set_title(0, '1982')
#tab.set_title(1, '2010')
#tab

# Let's get NDVI pixel values at all of the rasters at that point

In [ ]:
pixel_location = Point({"x" :-10093991.604, "y" : 4689459.491, "spatialReference" : {"wkid" : 102100}})

time = []
dtime = []
pixels = []

for idx,row in enumerate(im_sdf.iterrows()):


    oid = str(row[1]['OBJECTID'])
    
    image_at_t = ndvi_lyr.filter_by(where="OBJECTID = "+oid)#, geometry=the_geom)
    pixel = image_at_t.identify(geometry=pixel_location)#, time_extent=t)
    
    print(row[1]['Date'])

    try:
        pix = [float(x) for x in pixel['value'].split(',')]
        print(pix)
        pixels.append(pix)
        time.append(float(row[1]['AcquisitionDate']))
        dtime.append(row[1]['Date'])
    except:
        print("NoData")
    

In [ ]:
#import pickle

#pixels_pickle = open("ndvi_vals.p","wb")
#pickle.dump(pixels, pixels_pickle)
#pixels_pickle.close()

#dtime_pickle = open("dtime_vals.p","wb")
#pickle.dump(dtime, dtime_pickle)
#dtime_pickle.close()

#time_pickle = open("time_vals.p","wb")
#pickle.dump(time, time_pickle)
#time_pickle.close()

In [ ]:
import pickle

pixels = pickle.load(open("ndvi_vals.p","rb"))
dtime = pickle.load(open("dtime_vals.p","rb"))
time = pickle.load(open("time_vals.p","rb"))

# Let's put the QA Band values into it's own array

In [ ]:
qa_band = []
pixel_values = []
for pix in pixels:
    qa_band.append(pix[1])
    pixel_values.append(pix[0])

In [ ]:
qa_band

# Let's plot  the harmonic curve

We do this first without any filtering and it's hard to tell that there is a harmonic trend.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-darkgrid')

fig=plt.figure(figsize=(14, 6), dpi= 80, facecolor='w', edgecolor='k')

plt.plot(dtime, pixel_values, '*',label='linear') #indicde 0 is max
plt.xlabel('Time')
plt.ylabel('NDVI')

## Let's Filter the QA Bands

We can use the Landsat QA band to filter out snow, ice, and clouds. Here we wll apply that filter.

In [ ]:
LANDSAT_5_CLEAR_PIX_VALS = [672, 676, 680, 684]
QA_BAND_IND = 1

In [ ]:
clear_indices = [x for x in range(len(qa_band)) if qa_band[x] in LANDSAT_5_CLEAR_PIX_VALS]
clear_indices

# We can clear tis up if we sort the data

# Let's only use times and pixels that are "clear"

In [ ]:
clear_pix = [pixel_values[val] for val in  clear_indices]

clear_time = [time[x] for x in clear_indices]
clear_dtime = [dtime[x] for x in clear_indices]

In [ ]:
import numpy as np

sorted_t = np.sort(clear_dtime)
sorted_t_idx = np.argsort(clear_dtime)

sorted_clear_time = np.sort(clear_time)

sorted_clear_pix = [clear_pix[int(sorted_idx)] for sorted_idx in sorted_t_idx]

# Now, what does the plot look like?

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(14, 6), dpi= 80, facecolor='w', edgecolor='k')

plt.plot(clear_dtime, clear_pix, "*",label='linear') #indicde 0 is max
plt.xlabel('Time')
plt.ylabel('NDVI')

# Now we can start to see the trend

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(14, 6), dpi= 80, facecolor='w', edgecolor='k')

plt.plot(sorted_t, sorted_clear_pix,label='linear') #indicde 0 is max
plt.xlabel('Time')
plt.ylabel('NDVI')

# Something changed between 1996 and 1998

In [ ]:
date_df = im_sdf[im_sdf['CloudCover']<.10]
date_df[(date_df['Date'] > '1996-01-01') & (date_df['Date'] < '1997-01-01')]

In [ ]:
map_1996 = gis.map("122 Arabian Path, O'Fallon, MO")
map_1996

In [ ]:
selected_1996 = ms_lyr.filter_by(where="Name = 'LT50240331996272XXX02'")

map_1996.add_layer(stretch(extract_band(selected_1996,[2,1,0]),
                         stretch_type='StdDev',
                         num_stddev=2.5,
                         dra=True))

In [ ]:
map_1996.zoom = 14
map_1996.draw(geometry, symbol=symbol)

# 1997

In [ ]:
date_df = im_sdf[im_sdf['CloudCover']<.10]
date_df[(date_df['Date'] > '1997-01-01') & (date_df['Date'] < '1998-01-01')].Name

In [ ]:
map_1997 = gis.map("122 Arabian Path, O'Fallon, MO")
map_1997

In [ ]:
selected_1997 = ms_lyr.filter_by(where="Name = 'LT50240331997274XXX02'")

map_1997.add_layer(stretch(extract_band(selected_1997,[2,1,0]),
                         stretch_type='StdDev',
                         num_stddev=2.5,
                         dra=True))

In [ ]:
map_1997.zoom = 14
map_1997.draw(geometry, symbol=symbol)

# Trend Analysis

In [ ]:
import pandas as pd

df = pd.DataFrame()
df['Date'] = sorted_t[130:256]
df['vals'] = sorted_clear_pix[130:256]
df = df.set_index('Date')
df.index = pd.to_datetime(df.index)

In [ ]:
df

In [ ]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 18, 8
decomposition = sm.tsa.seasonal_decompose(df, freq=12)
fig = decomposition.plot()
plt.show()

https://medium.com/@josemarcialportilla/using-python-and-auto-arima-to-forecast-seasonal-time-series-90877adff03c

In [ ]:
decomposition.trend